all_data에서 healthy를 복사하는 경우.

phrase 데이터

In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn # 인공 신경망 모델들 모아놓은 모듈
import torch.nn.functional as F #그중 자주 쓰이는것들을 F로
from torchvision import transforms, datasets
import cv2
import pandas as pd
import os
from glob import glob
import torchvision.models as models
import sys
import librosa, librosa.display 

p = os.path.abspath('D:/project/voice_pathology_ai/model') # 상위 폴더를 사용하기 위해서.
sys.path.insert(1, p)
from pytorchtools.pytorchtools import EarlyStopping # 상위 폴더에 추가된 모듈.


if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
#DEVICE = torch.device('cpu')
print('Using Pytorch version : ',torch.__version__,' Device : ',DEVICE)

Using Pytorch version :  1.10.2  Device :  cuda


In [2]:
import torchaudio
import torchaudio.transforms as T

# 데이터 나누기 - Stratified KFold

- pathology : 1194 / healthy : 634 / 총 1828
- k = 5

## 1. test/ train 나누기


In [3]:
#1. train, test 나누기

import os
from glob import glob
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.model_selection import train_test_split # train , test 분리에 사용.


pathology = glob('D:/project/voice_pathology_ai/voice_data/all_data/pathology/phrase/*.wav')
healthy = glob('D:/project/voice_pathology_ai/voice_data/all_data/healthy/phrase/*.wav')
print("Pathology : ",len(pathology))
print("Healthy: ",len(healthy))

pathology= [ path.split("\\")[-1] for path in pathology] # path 데이터 변환.
healthy= [ path.split("\\")[-1] for path in healthy] # path 데이터 변환.
 # path 데이터 변환 #외부데이터로 가져오기위해서, 번호만 남긴다

    
X = pathology+healthy # path 데이터 합
print("총 데이터수 : ",len(X))
Y = [] # 라벨
for idx,x in enumerate(X):
    if idx<1354:
        Y.append("pathology")
    else:
        Y.append("healthy")

X, X_test, Y, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, stratify=Y, random_state=456)
#stratify를 넣어서, test에도 라벨별 잘 분류되게 한다.

print("---")
print("훈련 셋 : ",len(Y),Counter(Y))
print("테스트 셋 : ",len(Y_test),Counter(Y_test))
print("---")

Pathology :  1354
Healthy:  634
총 데이터수 :  1988
---
훈련 셋 :  1590 Counter({'pathology': 1083, 'healthy': 507})
테스트 셋 :  398 Counter({'pathology': 271, 'healthy': 127})
---


## 2. stratified k-fold

In [4]:
#1. train, test 나누기
#stratified kfold
import os
import random #데이터 shuffle 사용
from glob import glob
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from collections import Counter



skf_iris = StratifiedKFold(n_splits=5,shuffle=True,random_state=456)
cnt_iter = 0

X_train_list = [] #데이터 셋 보관
Y_train_list = []

X_valid_list = []
Y_valid_list = []

for train_idx, test_idx in skf_iris.split(X,Y):
    
    #split으로 반환된 인덱스를 이용하여, 학습 검증용 테스트 데이터 추출
    cnt_iter += 1
    X_train, X_valid = [X[idx] for idx in train_idx.tolist() ], [X[idx] for idx in test_idx.tolist() ]
    Y_train, Y_valid = [Y[idx] for idx in train_idx.tolist() ], [Y[idx] for idx in test_idx.tolist() ]
    
    X_train_list.append(X_train)
    X_valid_list.append(X_valid)
    
    Y_train_list.append(Y_train)
    Y_valid_list.append(Y_valid)
    
    
    #학습 및 예측
    
    label_train = Y_train
    label_test = Y_valid
    unique_train, train_counts = np.unique(label_train, return_counts = True)
    unique_test, test_counts = np.unique(label_test, return_counts = True)
    
    uniq_cnt_train = dict(zip(unique_train, train_counts))
    uniq_cnt_test = dict(zip(unique_test, test_counts))
    
    
    
    print('교차 검증 : {}'.format(cnt_iter))
    print('학습 레이블 데이터 분포 : \n', uniq_cnt_train)
    print('검증 레이블 데이터 분포 : \n', uniq_cnt_test,'\n')



교차 검증 : 1
학습 레이블 데이터 분포 : 
 {'healthy': 406, 'pathology': 866}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 217} 

교차 검증 : 2
학습 레이블 데이터 분포 : 
 {'healthy': 406, 'pathology': 866}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 217} 

교차 검증 : 3
학습 레이블 데이터 분포 : 
 {'healthy': 406, 'pathology': 866}
검증 레이블 데이터 분포 : 
 {'healthy': 101, 'pathology': 217} 

교차 검증 : 4
학습 레이블 데이터 분포 : 
 {'healthy': 405, 'pathology': 867}
검증 레이블 데이터 분포 : 
 {'healthy': 102, 'pathology': 216} 

교차 검증 : 5
학습 레이블 데이터 분포 : 
 {'healthy': 405, 'pathology': 867}
검증 레이블 데이터 분포 : 
 {'healthy': 102, 'pathology': 216} 



In [5]:
import pickle

 
#load
with open("D:/project/voice_pathology_ai/voice_data/all_data/phrase_sig_dict.pickle","rb") as fr:
    phrase_dict = pickle.load(fr)

    

# healthy spectro 출력

In [6]:

folds_healthy=[]
for fold in range(5):
    print("{}번 fold".format(fold))
    folds_healthy.append([ index for index,label in enumerate(Y_train_list[fold]) if label=='healthy' ])
    
    

0번 fold
1번 fold
2번 fold
3번 fold
4번 fold


In [42]:
#spectro 출력
import pickle

sr=16000
win_length =  np.int64(sr/40) # 
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.


for idx,fold in enumerate(folds_healthy):
    healthy_dict=dict()
    print("{}번 fold\n".format(idx))
    for index in fold:
        
        sig=phrase_dict[X_train_list[idx][index]]
        stft = librosa.stft(sig, win_length=win_length,n_fft=n_fft, hop_length=hop_length)
        
        
        
        healthy_dict[X_train_list[idx][index]]=stft
    
    
    
    ## Save pickle
    with open("D:/project/voice_pathology_ai/model/varauto/spectro/alldata/fold{}/healthy/healthy-stft.pickle".format(idx+1)
              ,"wb") as fw:
        pickle.dump(healthy_dict, fw)
        
    

0번 fold

1번 fold

2번 fold

3번 fold

4번 fold



# trains set mean, std 출력

In [18]:
X
Y

['pathology',
 'pathology',
 'pathology',
 'pathology',
 'pathology',
 'pathology',
 'healthy',
 'pathology',
 'pathology',
 'pathology',
 'pathology',
 'pathology',
 'pathology',
 'healthy',
 'pathology',
 'pathology',
 'healthy',
 'pathology',
 'pathology',
 'pathology',
 'pathology',
 'healthy',
 'pathology',
 'pathology',
 'pathology',
 'pathology',
 'pathology',
 'pathology',
 'pathology',
 'pathology',
 'pathology',
 'healthy',
 'pathology',
 'pathology',
 'pathology',
 'pathology',
 'pathology',
 'healthy',
 'healthy',
 'pathology',
 'pathology',
 'healthy',
 'healthy',
 'pathology',
 'pathology',
 'healthy',
 'healthy',
 'pathology',
 'pathology',
 'pathology',
 'pathology',
 'pathology',
 'healthy',
 'healthy',
 'pathology',
 'healthy',
 'pathology',
 'healthy',
 'pathology',
 'healthy',
 'healthy',
 'pathology',
 'healthy',
 'pathology',
 'pathology',
 'healthy',
 'pathology',
 'pathology',
 'pathology',
 'pathology',
 'healthy',
 'pathology',
 'pathology',
 'healthy',
 'path

In [61]:

train_healthy_ind=[]
train_healthy_ind+=[ index for index,label in enumerate(Y) if label=='healthy' ]

In [42]:
train_healthy_ind

[6,
 13,
 16,
 21,
 31,
 37,
 38,
 41,
 42,
 45,
 46,
 52,
 53,
 55,
 57,
 59,
 60,
 62,
 65,
 70,
 73,
 75,
 80,
 82,
 86,
 89,
 99,
 100,
 101,
 104,
 106,
 110,
 111,
 112,
 113,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 123,
 125,
 126,
 127,
 132,
 133,
 141,
 148,
 149,
 151,
 160,
 163,
 164,
 173,
 175,
 179,
 184,
 189,
 190,
 201,
 202,
 205,
 207,
 210,
 211,
 218,
 220,
 222,
 227,
 238,
 239,
 240,
 241,
 242,
 246,
 250,
 254,
 255,
 259,
 260,
 263,
 264,
 265,
 267,
 285,
 291,
 294,
 297,
 298,
 302,
 306,
 312,
 315,
 316,
 320,
 326,
 328,
 331,
 337,
 347,
 348,
 350,
 351,
 354,
 355,
 356,
 361,
 364,
 369,
 370,
 371,
 381,
 383,
 385,
 386,
 388,
 391,
 396,
 397,
 400,
 401,
 403,
 406,
 409,
 410,
 413,
 420,
 423,
 425,
 426,
 429,
 430,
 437,
 439,
 440,
 441,
 446,
 451,
 454,
 457,
 464,
 465,
 467,
 470,
 471,
 473,
 477,
 481,
 488,
 489,
 490,
 491,
 493,
 494,
 496,
 499,
 504,
 514,
 521,
 522,
 525,
 526,
 530,
 534,
 537,
 546,
 548,
 549,
 552,
 5

In [62]:
data_list=[]
for ind in train_healthy_ind:
    data_list.append(X[ind])

In [44]:
data_list

['69-phrase.wav',
 '25-phrase.wav',
 '1536-phrase.wav',
 '953-phrase.wav',
 '690-phrase.wav',
 '688-phrase.wav',
 '1505-phrase.wav',
 '1377-phrase.wav',
 '2261-phrase.wav',
 '11-phrase.wav',
 '1149-phrase.wav',
 '1181-phrase.wav',
 '1036-phrase.wav',
 '806-phrase.wav',
 '969-phrase.wav',
 '962-phrase.wav',
 '1211-phrase.wav',
 '1006-phrase.wav',
 '2285-phrase.wav',
 '1368-phrase.wav',
 '1848-phrase.wav',
 '1356-phrase.wav',
 '26-phrase.wav',
 '2561-phrase.wav',
 '1357-phrase.wav',
 '1916-phrase.wav',
 '1373-phrase.wav',
 '95-phrase.wav',
 '973-phrase.wav',
 '33-phrase.wav',
 '7-phrase.wav',
 '999-phrase.wav',
 '736-phrase.wav',
 '2576-phrase.wav',
 '77-phrase.wav',
 '1726-phrase.wav',
 '1104-phrase.wav',
 '1177-phrase.wav',
 '67-phrase.wav',
 '952-phrase.wav',
 '1865-phrase.wav',
 '1172-phrase.wav',
 '1738-phrase.wav',
 '679-phrase.wav',
 '991-phrase.wav',
 '941-phrase.wav',
 '1876-phrase.wav',
 '954-phrase.wav',
 '1544-phrase.wav',
 '680-phrase.wav',
 '1587-phrase.wav',
 '837-phrase.w

In [63]:
sr=16000
win_length =  np.int64(sr/40) # 
n_fft= win_length # WINDOWS SIZE중 사용할 길이. WINDOW SIZE가 넘어가면 나머지 것들은 zero padding
hop_length= np.int64( np.ceil(win_length/4) ) #  얼마만큼 시간 주기(sample)를 이동하면서 분석을 할 것인지. 일반적으로 window size의 1/4
#또는 10ms만큼으로 한다고 한다.

#sig to stft
healthy_stft_dict=dict()
for data_name in data_list:
    sig=phrase_dict[data_name]
    stft = librosa.stft(sig, win_length=win_length,n_fft=n_fft, hop_length=hop_length)
    magnitude = np.abs(stft)
    log_spectrogram = librosa.amplitude_to_db(magnitude)
    healthy_stft_dict[data_name]=log_spectrogram


In [64]:
length=300
pad2d = lambda a, i: a[:, 0:i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0], i-a.shape[1]))))
healthy_data=[ pad2d(healthy_stft_dict[data_name],length)[:128,:] for data_name in data_list]


In [65]:
np.array(healthy_data).mean()

-25.12305862766925

In [66]:
np.array(healthy_data).std()

17.943412194886722